# HOMO-LUMO Gap Predictions

### Problem Statement & Motivation

Accurately predicting quantum chemical properties like the HOMO–LUMO energy gap is essential for advancing materials science, drug discovery, and electronic design. The HOMO–LUMO gap is particularly informative for assessing molecular reactivity and stability. While Density Functional Theory (DFT) provides precise estimates, its high computational cost makes it impractical for large-scale screening of molecular libraries. This notebook explores machine learning alternatives that are fast, scalable, and interpretable, offering solutions that are accessible even on modest hardware.

### Related Work & Key Gap

Past work has shown that:

* DFT is accurate but computationally intensive
* ML models like kernel methods and GNNs show promise, but often require large models and expensive hardware

Key Gap: A need for lightweight, high-performing models that can run locally and integrate with user-friendly tools for deployment in research or education.

### Methodology & Evaluation

This notebook:

* Benchmarks a variety of 2D-based models using RDKit descriptors, Coulomb matrices, and graph neural networks (GNNs) on a 5k molecule subset
* Progresses to a hybrid GNN architecture combining OGB-standard graphs with SMILES-derived cheminformatics features
* Achieves **MAE = 0.159 eV**
* Visualizes results using parity plots, error inspection, and predicted-vs-true comparisons
* Evaluates both random and high-error cases to better understand model behavior

| Metric   | Best Model (Hybrid GNN) |
| -------- | ----------------------- |
| **MAE**  | 0.159 eV                |
| **RMSE** | 0.234 eV                |
| **R²**   | 0.965                   |


### Deployment & Accessibility

To make the model practically useful, an **interactive web app** was developed:

**Live App**: [HOMO–LUMO Gap Predictor on Hugging Face](https://huggingface.co/spaces/MooseML/homo-lumo-gap-predictor)

Features:

* **SMILES input** for any organic molecule
* **Real-time prediction** of the HOMO–LUMO gap
* **Molecular visualization**
* Simple **CSV logging** for result tracking

GitHub Repository: [MooseML/homo-lumo-gap-models](https://github.com/MooseML/homo-lumo-gap-models)


In [1]:
# general 
import pandas as pd
import numpy as np
from tqdm import tqdm
import ace_tools_open as tools
import optuna
import optuna.visualization as vis
import pickle
import joblib
import os 

# plotting 
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

# PyTorch
import torch
import torch.nn.functional as F
from torch.nn import Linear, ReLU, Module, Sequential, Dropout
from torch.utils.data import Subset
import torch.optim as optim
# PyTorch Geometric
from torch_geometric.nn import GINEConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from transformers import get_cosine_schedule_with_warmup

# OGB dataset 
from ogb.lsc import PygPCQM4Mv2Dataset, PCQM4Mv2Dataset
from ogb.utils import smiles2graph
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder

# RDKit
# from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit import Chem

# ChemML
from chemml.chem import Molecule, RDKitFingerprint, CoulombMatrix, tensorise_molecules
from chemml.models import MLP, NeuralGraphHidden, NeuralGraphOutput
from chemml.utils import regression_metrics

# SKlearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [2]:
print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("CUDA available:", tf.test.is_built_with_gpu_support())
print(tf.config.list_physical_devices('GPU'))
# list all GPUs
gpus = tf.config.list_physical_devices('GPU')

# check compute capability if GPU available
if gpus:
    for gpu in gpus:
        details = tf.config.experimental.get_device_details(gpu)
        print(f"Device: {gpu.name}")
        print(f"Compute Capability: {details.get('compute_capability')}")
else:
    print("No GPU found.")

TensorFlow version: 2.10.0
Built with CUDA: True
CUDA available: True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Device: /physical_device:GPU:0
Compute Capability: (8, 6)


In [3]:
# Paths - Fixed for Kaggle environment
if os.path.exists('/kaggle'):
    DATA_ROOT = '/kaggle/input/neurips-open-polymer-prediction-2025'
    CHUNK_DIR = '/kaggle/working/processed_chunks'  # Writable directory
    BACKBONE_PATH = '/kaggle/input/polymer/best_gnn_transformer_hybrid.pt'
else:
    DATA_ROOT = 'data'
    CHUNK_DIR = os.path.join(DATA_ROOT, 'processed_chunks')
    BACKBONE_PATH = 'best_gnn_transformer_hybrid.pt'

TRAIN_LMDB = os.path.join(CHUNK_DIR, 'polymer_train3d_dist.lmdb')
TEST_LMDB = os.path.join(CHUNK_DIR, 'polymer_test3d_dist.lmdb')

print(f"Data root: {DATA_ROOT}")
print(f"LMDB directory: {CHUNK_DIR}")
print(f"Train LMDB: {TRAIN_LMDB}")
print(f"Test LMDB: {TEST_LMDB}")

# Create LMDBs if they don't exist
if not os.path.exists(TRAIN_LMDB) or not os.path.exists(TEST_LMDB):
    print('Building LMDBs...')
    os.makedirs(CHUNK_DIR, exist_ok=True)
    # Run the LMDB builders
    !python build_polymer_lmdb_fixed.py train
    !python build_polymer_lmdb_fixed.py test
    print('LMDB creation complete.')
else:
    print('LMDBs already exist.')


Data root: data
LMDB directory: data\processed_chunks
Train LMDB: data\processed_chunks\polymer_train3d_dist.lmdb
Test LMDB: data\processed_chunks\polymer_test3d_dist.lmdb
LMDBs already exist.


In [4]:
# LMDB+CSV wiring 
import os, numpy as np, pandas as pd

# 1) Columns / index mapping
label_cols = ['Tg','FFV','Tc','Density','Rg']
task2idx   = {k:i for i,k in enumerate(label_cols)}

# 2) Read the training labels (CSV is only used to know which IDs have labels)
train_path = os.path.join(DATA_ROOT, 'train.csv')
train_df   = pd.read_csv(train_path)
assert {'id','SMILES'}.issubset(train_df.columns), "train.csv must have id and SMILES"
train_df['id'] = train_df['id'].astype(int)

# 3) Read the actual IDs that exist in the LMDB
def read_lmdb_ids(lmdb_path: str) -> np.ndarray:
    ids_txt = lmdb_path + ".ids.txt"
    if not os.path.exists(ids_txt):
        raise FileNotFoundError(f"Missing {ids_txt}. Rebuild LMDB or confirm paths.")
    ids = np.loadtxt(ids_txt, dtype=np.int64)
    if ids.ndim == 0:  # single id edge case
        ids = ids.reshape(1)
    return ids

lmdb_ids = read_lmdb_ids(TRAIN_LMDB)
print(f"LMDB contains {len(lmdb_ids):,} train graphs")

# 4) Helper: IDs that have a label for a given task (intersection with LMDB ids)
def ids_with_label(task: str) -> np.ndarray:
    col = task
    have_label = train_df.loc[~train_df[col].isna(), 'id'].astype(int).values
    # Only keep those that were actually written to the LMDB
    keep = np.intersect1d(have_label, lmdb_ids, assume_unique=False)
    return keep

# 5) Make a global pool split once (reused for each task)
rng = np.random.default_rng(123)
perm = rng.permutation(len(lmdb_ids))
split = int(0.9 * len(lmdb_ids))
train_pool_ids = lmdb_ids[perm[:split]]
val_pool_ids   = lmdb_ids[perm[split:]]

print(f"Global pools -> train_pool={len(train_pool_ids):,}  val_pool={len(val_pool_ids):,}")

# 6) Quick sanity: show available counts per task
for t in label_cols:
    n_task_ids = len(ids_with_label(t))
    print(f"{t:>7}: {n_task_ids:6d} rows with labels (pre-intersection with pools)")


LMDB contains 7,973 train graphs
Global pools -> train_pool=7,175  val_pool=798
     Tg:    511 rows with labels (pre-intersection with pools)
    FFV:   7030 rows with labels (pre-intersection with pools)
     Tc:    737 rows with labels (pre-intersection with pools)
Density:    613 rows with labels (pre-intersection with pools)
     Rg:    614 rows with labels (pre-intersection with pools)


The only property that appears will succeed with a simple imputation strategy is FFV. All other properties contain very high percent missing. Therefore, I will impute median for FFV, train a model for FFV, and train separate models for other properties. I will attempt to filter out missing values for each property. If this yields uncessful, I may explore sampling techniques or use the trained model to impute values to train a secondaery model. |

In [5]:
import numpy as np, torch
from typing import List
from torch.utils.data import Dataset

def _safe_numpy(x, default_shape=None, dtype=np.float32):
    try:
        return torch.as_tensor(x).detach().cpu().numpy().astype(dtype)
    except Exception:
        if default_shape is None:
            return np.array([], dtype=dtype)
        return np.zeros(default_shape, dtype=dtype)

def geom_features_from_rec(rec, rdkit_dim_expected=15, rbf_K=32) -> np.ndarray:
    """
    Build a fixed-length vector from a single LMDB record:
      [rdkit(15), n_atoms, n_bonds, deg_mean, deg_max, has_xyz,
       eig3(3), bbox_extents(3), radius_stats(3), hop_hist(3), extra_atom_mean(5),
       edge_rbf_mean(32)]
    ~ total len = 15 + 5 + 3 + 3 + 3 + 3 + 5 + 32 = 69
    """
    # 15 RDKit descriptors stored in LMDB (your rebuilt version)
    rd = getattr(rec, "rdkit_feats", None)
    rd = _safe_numpy(rd, default_shape=(1, rdkit_dim_expected)).reshape(-1)
    if rd.size != rdkit_dim_expected:
        rd = np.zeros((rdkit_dim_expected,), dtype=np.float32)

    # basic graph sizes & degree
    x = torch.as_tensor(rec.x)             # [N, ...]
    ei = torch.as_tensor(rec.edge_index)   # [2, E]
    n = x.shape[0]
    e = ei.shape[1] if ei.ndim == 2 else 0
    deg = torch.bincount(ei[0], minlength=n) if e > 0 else torch.zeros(n, dtype=torch.long)
    deg_mean = deg.float().mean().item() if n > 0 else 0.0
    deg_max  = deg.max().item() if n > 0 else 0.0

    # has_xyz flag
    has_xyz = int(bool(getattr(rec, "has_xyz", torch.zeros(1, dtype=torch.bool))[0].item())) if hasattr(rec, "has_xyz") else 0

    # pos-based features
    eig3 = np.zeros(3, dtype=np.float32)
    extents = np.zeros(3, dtype=np.float32)
    rad_stats = np.zeros(3, dtype=np.float32)
    pos = getattr(rec, "pos", None)
    if pos is not None and n > 0 and has_xyz:
        P = torch.as_tensor(pos).float()                     # [N,3]
        center = P.mean(dim=0, keepdim=True)
        C = P - center
        cov = (C.T @ C) / max(1, n-1)                       # [3,3]
        vals = torch.linalg.eigvalsh(cov).clamp_min(0).sqrt()  # length scales
        eig3 = vals.detach().cpu().numpy()
        mn, mx = P.min(0).values, P.max(0).values
        extents = (mx - mn).detach().cpu().numpy()
        r = C.norm(dim=1)
        rad_stats = np.array([r.mean().item(), r.std().item(), r.max().item()], dtype=np.float32)

    # hop-distance histogram (1,2,3 hops)
    hop_hist = np.zeros(3, dtype=np.float32)
    D = getattr(rec, "dist", None)
    if D is not None and n > 0:
        Dn = torch.as_tensor(D).float()[:n, :n]
        hop_hist = np.array([
            (Dn == 1).float().mean().item(),
            (Dn == 2).float().mean().item(),
            (Dn == 3).float().mean().item()
        ], dtype=np.float32)

    # extra atom features (mean over atoms, 5 dims if present)
    extra_atom = getattr(rec, "extra_atom_feats", None)
    extra_mean = np.zeros(5, dtype=np.float32)
    if extra_atom is not None and hasattr(extra_atom, "shape") and extra_atom.shape[-1] == 5:
        extra_mean = torch.as_tensor(extra_atom).float().mean(dim=0).detach().cpu().numpy()

    # edge RBF (last 32 channels of edge_attr were RBF(d))
    rbf_mean = np.zeros(rbf_K, dtype=np.float32)
    ea = getattr(rec, "edge_attr", None)
    if ea is not None:
        EA = torch.as_tensor(ea)
        if EA.ndim == 2 and EA.shape[1] >= (3 + rbf_K):
            rbf = EA[:, -rbf_K:].float()
            rbf_mean = rbf.mean(dim=0).detach().cpu().numpy()

    scalars = np.array([n, e, deg_mean, deg_max, has_xyz], dtype=np.float32)
    return np.concatenate([rd, scalars, eig3, extents, rad_stats, hop_hist, extra_mean, rbf_mean], axis=0)


In [6]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors as rdmd, DataStructs
from dataset_polymer_fixed import LMDBDataset

def morgan_bits(smiles_list, n_bits=1024, radius=3):
    X = np.zeros((len(smiles_list), n_bits), dtype=np.uint8)
    for i, s in enumerate(smiles_list):
        arr = np.zeros((n_bits,), dtype=np.uint8)
        m = Chem.MolFromSmiles(s)
        if m is not None:
            fp = rdmd.GetMorganFingerprintAsBitVect(m, radius=radius, nBits=n_bits)
            DataStructs.ConvertToNumpyArray(fp, arr)
        X[i] = arr
    return X.astype(np.float32)

def build_rf_features_from_lmdb(ids: np.ndarray, lmdb_path: str, smiles_list: List[str]) -> np.ndarray:
    """
    Returns X = [Morgan1024 | LMDB-3D-global(69)] for each id/smiles.
    Assumes ids and smiles_list are aligned with the CSV used to build LMDB.
    """
    base = LMDBDataset(ids, lmdb_path)
    # 3D/global block
    feats3d = []
    for i in range(len(base)):
        rec = base[i]
        feats3d.append(geom_features_from_rec(rec))  # shape (69,)
    X3d = np.vstack(feats3d).astype(np.float32) if feats3d else np.zeros((0, 69), dtype=np.float32)

    # Morgan FP block (2D)
    Xfp = morgan_bits(smiles_list, n_bits=1024, radius=3)   # (N,1024)

    # concat
    X = np.hstack([Xfp, X3d]).astype(np.float32)            # (N, 1024+69)
    return X


# Models

In [7]:
# Use the CSV only to know which rows have labels; keep 'id' here.
train_df = pd.read_csv(os.path.join(DATA_ROOT, "train.csv"))
train_df["id"] = train_df["id"].astype(int)

def build_target_df_from_ids(df: pd.DataFrame, target_col: str, keep_ids: np.ndarray):
    """
    Return DataFrame with only SMILES + target, restricted to IDs present in the LMDB
    and dropping missing targets.
    """
    out = df.loc[df["id"].isin(keep_ids), ["SMILES", target_col]].copy()
    print(f"Initial {target_col} shape:", out.shape)
    print(f"Initial {target_col} missing:\n{out.isnull().sum()}")
    out = out.dropna(subset=[target_col]).reset_index(drop=True)
    print(f"Cleaned {target_col} shape:", out.shape)
    print(f"Cleaned {target_col} missing:\n{out.isnull().sum()}\n")
    return out

# Build all five (use same LMDB id set so we only keep rows that exist in LMDB)
df_tg      = build_target_df_from_ids(train_df, "Tg",      lmdb_ids)
df_density = build_target_df_from_ids(train_df, "Density", lmdb_ids)
df_ffv     = build_target_df_from_ids(train_df, "FFV",     lmdb_ids)
df_tc      = build_target_df_from_ids(train_df, "Tc",      lmdb_ids)
df_rg      = build_target_df_from_ids(train_df, "Rg",      lmdb_ids)


Initial Tg shape: (7973, 2)
Initial Tg missing:
SMILES       0
Tg        7462
dtype: int64
Cleaned Tg shape: (511, 2)
Cleaned Tg missing:
SMILES    0
Tg        0
dtype: int64

Initial Density shape: (7973, 2)
Initial Density missing:
SMILES        0
Density    7360
dtype: int64
Cleaned Density shape: (613, 2)
Cleaned Density missing:
SMILES     0
Density    0
dtype: int64

Initial FFV shape: (7973, 2)
Initial FFV missing:
SMILES      0
FFV       943
dtype: int64
Cleaned FFV shape: (7030, 2)
Cleaned FFV missing:
SMILES    0
FFV       0
dtype: int64

Initial Tc shape: (7973, 2)
Initial Tc missing:
SMILES       0
Tc        7236
dtype: int64
Cleaned Tc shape: (737, 2)
Cleaned Tc missing:
SMILES    0
Tc        0
dtype: int64

Initial Rg shape: (7973, 2)
Initial Rg missing:
SMILES       0
Rg        7359
dtype: int64
Cleaned Rg shape: (614, 2)
Cleaned Rg missing:
SMILES    0
Rg        0
dtype: int64



In [8]:
# Morgan FP utilities (no 3D, no external descriptors) 
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import numpy as np
from typing import Optional, Tuple
from tqdm.auto import tqdm

def smiles_to_morgan_fp(
    smi: str,
    n_bits: int = 1024,
    radius: int = 3,
    use_counts: bool = False,
) -> Optional[np.ndarray]:
    """Return a 1D numpy array Morgan fingerprint; None if SMILES invalid."""
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None
    if use_counts:
        fp = rdMolDescriptors.GetMorganFingerprint(mol, radius)
        # convert to dense count vector
        arr = np.zeros((n_bits,), dtype=np.int32)
        for bit_id, count in fp.GetNonzeroElements().items():
            arr[bit_id % n_bits] += count
        return arr.astype(np.float32)
    else:
        bv = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        arr = np.zeros((n_bits,), dtype=np.int8)
        Chem.DataStructs.ConvertToNumpyArray(bv, arr)
        return arr.astype(np.float32)

def prepare_fp_for_target(
    df_target: pd.DataFrame,
    target_col: str,
    *,
    fp_bits: int = 1024,
    fp_radius: int = 3,
    use_counts: bool = False,
    save_csv_path: Optional[str] = None,
    show_progress: bool = True,
) -> Tuple[pd.DataFrame, np.ndarray, np.ndarray]:
    """
    Drop missing targets, compute Morgan FPs from SMILES only.
    Returns (df_clean, y, X_fp) where:
      df_clean: ['SMILES', target_col]
      y: (N,)
      X_fp: (N, fp_bits)
    """
    assert {"SMILES", target_col}.issubset(df_target.columns)

    # 1) drop missing targets (no imputation)
    work = df_target[["SMILES", target_col]].copy()
    before = len(work)
    work = work.dropna(subset=[target_col]).reset_index(drop=True)
    after = len(work)
    print(f"[{target_col}] dropped {before - after} missing; kept {after}")

    # 2) compute FPs; skip invalid SMILES
    fps, ys, keep_smiles = [], [], []
    it = work.itertuples(index=False)
    if show_progress:
        it = tqdm(it, total=len(work), desc=f"FPs for {target_col}")

    for row in it:
        smi = row.SMILES
        yv  = getattr(row, target_col)
        arr = smiles_to_morgan_fp(smi, n_bits=fp_bits, radius=fp_radius, use_counts=use_counts)
        if arr is None:
            continue
        fps.append(arr)
        ys.append(float(yv))
        keep_smiles.append(smi)

    X_fp = np.stack(fps, axis=0) if fps else np.zeros((0, fp_bits), dtype=np.float32)
    y = np.asarray(ys, dtype=float)
    df_clean = pd.DataFrame({"SMILES": keep_smiles, target_col: y})

    if save_csv_path:
        df_clean.to_csv(save_csv_path, index=False)
        print(f"[{target_col}] saved cleaned CSV -> {save_csv_path}")

    print(f"[{target_col}] X_fp: {X_fp.shape} | y: {y.shape}")
    return df_clean, y, X_fp


In [9]:
# Bit vectors (1024, r=3) 
df_clean_tg,      y_tg,      X_tg      = prepare_fp_for_target(df_tg,      "Tg",      fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_tg_fp.csv")
df_clean_density, y_density, X_density = prepare_fp_for_target(df_density, "Density", fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_density_fp.csv")
df_clean_ffv,     y_ffv,     X_ffv     = prepare_fp_for_target(df_ffv,     "FFV",     fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_ffv_fp.csv")
df_clean_tc,      y_tc,      X_tc      = prepare_fp_for_target(df_tc,      "Tc",      fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_tc_fp.csv")
df_clean_rg,      y_rg,      X_rg      = prepare_fp_for_target(df_rg,      "Rg",      fp_bits=1024, fp_radius=3, use_counts=False, save_csv_path="cleaned_rg_fp.csv")


[Tg] dropped 0 missing; kept 511


FPs for Tg:   0%|          | 0/511 [00:00<?, ?it/s]

[Tg] saved cleaned CSV -> cleaned_tg_fp.csv
[Tg] X_fp: (511, 1024) | y: (511,)
[Density] dropped 0 missing; kept 613


FPs for Density:   0%|          | 0/613 [00:00<?, ?it/s]

[Density] saved cleaned CSV -> cleaned_density_fp.csv
[Density] X_fp: (613, 1024) | y: (613,)
[FFV] dropped 0 missing; kept 7030


...

[FFV] saved cleaned CSV -> cleaned_ffv_fp.csv
[FFV] X_fp: (7030, 1024) | y: (7030,)
[Tc] dropped 0 missing; kept 737


FPs for Tc:   0%|          | 0/737 [00:00<?, ?it/s]

[Tc] saved cleaned CSV -> cleaned_tc_fp.csv
[Tc] X_fp: (737, 1024) | y: (737,)
[Rg] dropped 0 missing; kept 614


FPs for Rg:   0%|          | 0/614 [00:00<?, ?it/s]

[Rg] saved cleaned CSV -> cleaned_rg_fp.csv
[Rg] X_fp: (614, 1024) | y: (614,)


In [10]:
from dataclasses import dataclass
from typing import Optional, Tuple
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

@dataclass
class TabularSplits:
    # unscaled (for RF)
    X_train: np.ndarray
    X_test:  np.ndarray
    y_train: np.ndarray
    y_test:  np.ndarray
    # scaled (for KRR/MLP)
    X_train_scaled: Optional[np.ndarray] = None
    X_test_scaled:  Optional[np.ndarray] = None
    y_train_scaled: Optional[np.ndarray] = None  # shape (N,1)
    y_test_scaled:  Optional[np.ndarray] = None
    x_scaler: Optional[StandardScaler] = None
    y_scaler: Optional[StandardScaler] = None

def _make_regression_stratify_bins(y: np.ndarray, n_bins: int = 10) -> np.ndarray:
    """Return integer bins for approximate stratification in regression."""
    y = y.ravel()
    # handle degenerate case
    if np.unique(y).size < n_bins:
        n_bins = max(2, np.unique(y).size)
    quantiles = np.linspace(0, 1, n_bins + 1)
    bins = np.unique(np.quantile(y, quantiles))
    # ensure strictly increasing
    bins = np.unique(bins)
    # np.digitize expects right-open intervals by default
    strat = np.digitize(y, bins[1:-1], right=False)
    return strat

def make_tabular_splits(
    X: np.ndarray,
    y: np.ndarray,
    *,
    test_size: float = 0.2,
    random_state: int = 42,
    scale_X: bool = True,
    scale_y: bool = True,
    stratify_regression: bool = False,
    n_strat_bins: int = 10,
    # if you already decided splits (e.g., scaffold split), pass indices:
    train_idx: Optional[np.ndarray] = None,
    test_idx: Optional[np.ndarray] = None,
) -> TabularSplits:
    """
    Split and (optionally) scale tabular features/targets for a single target.
    Returns both scaled and unscaled arrays, plus fitted scalers.
    """
    y = np.asarray(y, dtype=float).ravel()
    X = np.asarray(X)

    if train_idx is not None and test_idx is not None:
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
    else:
        strat = None
        if stratify_regression:
            strat = _make_regression_stratify_bins(y, n_bins=n_strat_bins)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=strat
        )

    # Unscaled outputs (for RF, tree models)
    splits = TabularSplits(
        X_train=X_train, X_test=X_test,
        y_train=y_train, y_test=y_test
    )

    # Scaled versions (for KRR/MLP)
    if scale_X:
        xscaler = StandardScaler()
        splits.X_train_scaled = xscaler.fit_transform(X_train)
        splits.X_test_scaled  = xscaler.transform(X_test)
        splits.x_scaler = xscaler
    if scale_y:
        yscaler = StandardScaler()
        splits.y_train_scaled = yscaler.fit_transform(y_train.reshape(-1, 1))
        splits.y_test_scaled  = yscaler.transform(y_test.reshape(-1, 1))
        splits.y_scaler = yscaler

    # Shapes summary
    print("Splits:")
    print("X_train:", splits.X_train.shape, "| X_test:", splits.X_test.shape)
    if splits.X_train_scaled is not None:
        print("X_train_scaled:", splits.X_train_scaled.shape, "| X_test_scaled:", splits.X_test_scaled.shape)
    print("y_train:", splits.y_train.shape, "| y_test:", splits.y_test.shape)
    if splits.y_train_scaled is not None:
        print("y_train_scaled:", splits.y_train_scaled.shape, "| y_test_scaled:", splits.y_test_scaled.shape)

    return splits

In [11]:
from typing import Dict, Any, Tuple
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import numpy as np
import os

def train_eval_rf(
    X: np.ndarray,
    y: np.ndarray,
    *,
    rf_params: Dict[str, Any],
    test_size: float = 0.2,
    random_state: int = 42,
    stratify_regression: bool = True,
    n_strat_bins: int = 10,
    save_dir: str = "saved_models/rf",
    tag: str = "model",
) -> Tuple[RandomForestRegressor, Dict[str, float], TabularSplits, str]:
    """
    Trains a RandomForest on unscaled features; returns (model, metrics, splits, path).
    """
    os.makedirs(save_dir, exist_ok=True)
    # Pick a safe number of bins based on dataset size
    if stratify_regression:
        adaptive_bins = min(n_strat_bins, max(3, int(np.sqrt(len(y)))))
    else:
        adaptive_bins = n_strat_bins
    splits = make_tabular_splits(
        X, y,
        test_size=test_size,
        random_state=random_state,
        scale_X=False, scale_y=False,                 # RF doesn't need scaling
        stratify_regression=stratify_regression,
        n_strat_bins=adaptive_bins
    )

    rf = RandomForestRegressor(random_state=random_state, n_jobs=-1, **rf_params)
    rf.fit(splits.X_train, splits.y_train)

    pred_tr = rf.predict(splits.X_train)
    pred_te = rf.predict(splits.X_test)

    metrics = {
        "train_MAE": mean_absolute_error(splits.y_train, pred_tr),
        "train_RMSE": mean_squared_error(splits.y_train, pred_tr, squared=False),
        "train_R2": r2_score(splits.y_train, pred_tr),
        "val_MAE": mean_absolute_error(splits.y_test, pred_te),
        "val_RMSE": mean_squared_error(splits.y_test, pred_te, squared=False),
        "val_R2": r2_score(splits.y_test, pred_te),
    }
    print(f"[RF/{tag}] val_MAE={metrics['val_MAE']:.6f}  val_RMSE={metrics['val_RMSE']:.6f}  val_R2={metrics['val_R2']:.4f}")

    path = os.path.join(save_dir, f"rf_{tag}.joblib")
    joblib.dump({"model": rf, "metrics": metrics, "rf_params": rf_params}, path)
    return rf, metrics, splits, path

In [12]:
rf_cfg = {
    "FFV": {"n_estimators": 100, "max_depth": 60},
    "Tc":  {'n_estimators': 800, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False},
    "Rg":  {'n_estimators': 400, 'max_depth': 260, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 1.0, 'bootstrap': True},
}

rf_ffv, m_ffv, splits_ffv, p_ffv = train_eval_rf(X_ffv, y_ffv, rf_params=rf_cfg["FFV"], tag="FFV")
rf_tc,  m_tc,  splits_tc,  p_tc  = train_eval_rf(X_tc,  y_tc,  rf_params=rf_cfg["Tc"],  tag="Tc")
rf_rg,  m_rg,  splits_rg,  p_rg  = train_eval_rf(X_rg,  y_rg,  rf_params=rf_cfg["Rg"],  tag="Rg")
rf_tg,  m_tg,  splits_tg,  p_tg  = train_eval_rf(X_tg,  y_tg,  rf_params=rf_cfg["Rg"],  tag="Tg")
rf_density,  m_density,  splits_density,  p_density  = train_eval_rf(X_density,  y_density,  rf_params=rf_cfg["Rg"],  tag="Density")

Splits:
X_train: (5624, 1024) | X_test: (1406, 1024)
y_train: (5624,) | y_test: (1406,)
[RF/FFV] val_MAE=0.009095  val_RMSE=0.019753  val_R2=0.5701
Splits:
X_train: (589, 1024) | X_test: (148, 1024)
y_train: (589,) | y_test: (148,)
[RF/Tc] val_MAE=0.029866  val_RMSE=0.045109  val_R2=0.7304
Splits:
X_train: (491, 1024) | X_test: (123, 1024)
y_train: (491,) | y_test: (123,)
[RF/Rg] val_MAE=1.715067  val_RMSE=2.664982  val_R2=0.6916
Splits:
X_train: (408, 1024) | X_test: (103, 1024)
y_train: (408,) | y_test: (103,)
[RF/Tg] val_MAE=61.738193  val_RMSE=78.750171  val_R2=0.5333
Splits:
X_train: (490, 1024) | X_test: (123, 1024)
y_train: (490,) | y_test: (123,)
[RF/Density] val_MAE=0.054697  val_RMSE=0.092855  val_R2=0.6311


In [13]:
# === helpers (uses the LMDB feature builders you already added) ===
def train_rf_aug3d_for_target(
    target_col: str,
    rf_params: dict,
    *,
    train_csv_path: str,
    lmdb_path: str,
    save_dir: str = "saved_models/rf_aug3d",
    tag_prefix: str = "aug3D",
    test_size: float = 0.2,
    random_state: int = 42,
    stratify_regression: bool = True,
    n_strat_bins: int = 10,
):
    """Load rows with target, build X=[FP|3D], train RF via your train_eval_rf()."""
    df = pd.read_csv(train_csv_path)
    mask = ~df[target_col].isna()
    ids_tr    = df.loc[mask, 'id'].astype(int).values
    smiles_tr = df.loc[mask, 'SMILES'].astype(str).tolist()
    y         = df.loc[mask, target_col].astype(float).values

    X_aug = build_rf_features_from_lmdb(ids_tr, lmdb_path, smiles_tr)  # (N, 1024 + 69)

    model, metrics, splits, path = train_eval_rf(
        X_aug, y,
        rf_params=rf_params,
        test_size=test_size,
        random_state=random_state,
        stratify_regression=stratify_regression,
        n_strat_bins=n_strat_bins,
        save_dir=save_dir,
        tag=f"{target_col}_{tag_prefix}"
    )
    return model, metrics, splits, path

# === per-target configs (start with what worked; tweak later) ===
rf_cfg_aug = {
    "FFV":     {"n_estimators": 800, "max_depth": 30, "min_samples_leaf": 1, "max_features": "sqrt"},
    "Tc":      {'n_estimators': 800, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False},
    "Rg":      {'n_estimators': 400, 'max_depth': 260, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 1.0, 'bootstrap': True},
    # reasonable first passes for the two GNN targets (just to A/B):
    "Tg":      {"n_estimators": 600, "max_depth": 60, "min_samples_leaf": 1, "max_features": "sqrt"},
    "Density": {"n_estimators": 600, "max_depth": 40, "min_samples_leaf": 1, "max_features": "sqrt"},
}

# === train all five with augmented features ===
TRAIN_CSV = os.path.join(DATA_ROOT, "train.csv")
rf_models, rf_metrics, rf_splits, rf_paths = {}, {}, {}, {}

for t in ["FFV", "Tc", "Rg", "Tg", "Density"]:
    print(f"\n>>> Training RF(+3D) for {t}")
    m, met, sp, p = train_rf_aug3d_for_target(
        t, rf_cfg_aug[t],
        train_csv_path=TRAIN_CSV,
        lmdb_path=TRAIN_LMDB,
        save_dir="saved_models/rf_aug3d",
        tag_prefix="aug3D",
        test_size=0.2,
        random_state=42,
        stratify_regression=True,
        n_strat_bins=10,
    )
    rf_models[t], rf_metrics[t], rf_splits[t], rf_paths[t] = m, met, sp, p
    print(f"[RF+3D/{t}]  val_MAE={met['val_MAE']:.6f}  val_RMSE={met['val_RMSE']:.6f}  val_R2={met['val_R2']:.4f}")



>>> Training RF(+3D) for FFV
Splits:
X_train: (5624, 1093) | X_test: (1406, 1093)
y_train: (5624,) | y_test: (1406,)
[RF/FFV_aug3D] val_MAE=0.007621  val_RMSE=0.017553  val_R2=0.6605
[RF+3D/FFV]  val_MAE=0.007621  val_RMSE=0.017553  val_R2=0.6605

>>> Training RF(+3D) for Tc
Splits:
X_train: (589, 1093) | X_test: (148, 1093)
y_train: (589,) | y_test: (148,)
[RF/Tc_aug3D] val_MAE=0.029937  val_RMSE=0.045036  val_R2=0.7313
[RF+3D/Tc]  val_MAE=0.029937  val_RMSE=0.045036  val_R2=0.7313

>>> Training RF(+3D) for Rg
Splits:
X_train: (491, 1093) | X_test: (123, 1093)
y_train: (491,) | y_test: (123,)
[RF/Rg_aug3D] val_MAE=1.648818  val_RMSE=2.493712  val_R2=0.7299
[RF+3D/Rg]  val_MAE=1.648818  val_RMSE=2.493712  val_R2=0.7299

>>> Training RF(+3D) for Tg
Splits:
X_train: (408, 1093) | X_test: (103, 1093)
y_train: (408,) | y_test: (103,)
[RF/Tg_aug3D] val_MAE=58.315801  val_RMSE=74.296699  val_R2=0.5846
[RF+3D/Tg]  val_MAE=58.315801  val_RMSE=74.296699  val_R2=0.5846

>>> Training RF(+3D) for

In [14]:
# test_df  = pd.read_csv(os.path.join(DATA_ROOT, 'test.csv'))
# test_ids = test_df['id'].astype(int).values
# test_smiles = test_df['SMILES'].astype(str).tolist()

# X_test_aug = build_rf_features_from_lmdb(test_ids, TEST_LMDB, test_smiles)  # (M, 1093)

# # Example: swap in RF(+3D) for FFV and Tc (and optionally Tg/Density/Rg if better)
# rf_ffv = joblib.load(rf_paths["FFV"])['model']
# rf_tc  = joblib.load(rf_paths["Tc"])['model']

# pred_ffv = rf_ffv.predict(X_test_aug).astype(float)
# pred_tc  = rf_tc.predict(X_test_aug).astype(float)

# # If you want to try RF(+3D) for Tg/Density/Rg too:
# # pred_tg = joblib.load(rf_paths["Tg"])['model'].predict(X_test_aug).astype(float)
# # pred_density = joblib.load(rf_paths["Density"])['model'].predict(X_test_aug).astype(float)
# # pred_rg = joblib.load(rf_paths["Rg"])['model'].predict(X_test_aug).astype(float)


## ChemML GNN Model Results
| Model Type             | Featurization        |   MAE |  RMSE |   R² | Notes             |
|------------------------|----------------------|-------|-------|------|-------------------|
| GNN (Tuned)            | tensorise_molecules Graph   | 0.302 | 0.411 | 0.900 | Best performance across all metrics   |
| GNN (Untuned)          | tensorise_molecules Graph   | 0.400 | 0.519 | 0.841 | Good overall|


---
# Final Model Training

Having explored different molecular graph representations and model architectures, I am now moving to training what is expected to be the best-performing model using the full dataset. The earlier GNN model was based on `tensorise_molecules` (ChemML) graphs and had strong performance with a **mean absolute error (MAE) around 0.30**. These graphs are based on RDKit's internal descriptors and do not reflect the original PCQM4Mv2 graph structure used in the Open Graph Benchmark (OGB). Therefore, I will shift focus to the `smiles2graph` representation provided by OGB, which aligns more directly with the benchmark's evaluation setup and top-performing models on the leaderboard.


| Source                         | Atom/Bond Features                                                 | Format                                          | Customizable?     | Alignment with PCQM4Mv2?  |
| ------------------------------ | ------------------------------------------------------------------ | ----------------------------------------------- | ----------------- | ---------------------- |
| `tensorise_molecules` (ChemML) | RDKit-based descriptors (ex: atom number, degree, hybridization) | NumPy tensors (`X_atoms`, `X_bonds`, `X_edges`) | Limited           |  Not aligned          |
| `smiles2graph` (OGB / PyG)     | Predefined categorical features from PCQM4Mv2                      | PyTorch Geometric `Data` objects                |  Highly flexible |  Matches OGB standard |

By using `smiles2graph`, we:

* Use OGB-standard graph construction and feature encoding for fair comparisons with leaderboard models
* Include learnable AtomEncoder and BondEncoder embeddings from `ogb.graphproppred.mol_encoder`, which improve model expressiveness
* Maintain compatibility with PyTorch Geometric, DGL, and OGB tools

I will also concatenate GNN-derived embeddings with SMILES-based RDKit descriptors, feeding this hybrid representation into MLP head. This allows you to combine structural and cheminformatics perspectives for improved prediction accuracy. With this setup, I aim to improve upon the MAE of \~0.30 achieved earlier and push closer toward state-of-the-art performance.




In [15]:
label_cols = ['Tg','FFV','Tc','Density','Rg']
task2idx   = {k:i for i,k in enumerate(label_cols)}

train_csv = pd.read_csv(os.path.join(DATA_ROOT, "train.csv"))  # keep 'id'!
lmdb_ids_path = TRAIN_LMDB + ".ids.txt"
if os.path.exists(lmdb_ids_path):
    with open(lmdb_ids_path) as f:
        kept_ids = set(int(x.strip()) for x in f if x.strip())
else:
    kept_ids = set(train_csv['id'].astype(int).tolist())

def ids_for_task(task):
    t = task2idx[task]
    col = label_cols[t]
    ids = train_csv.loc[~train_csv[col].isna(), 'id'].astype(int).tolist()
    # only those that actually exist in LMDB
    return np.array([i for i in ids if i in kept_ids], dtype=int)

ids_tg  = ids_for_task("Tg")
ids_den = ids_for_task("Density")
ids_tc = ids_for_task("Tc")
ids_rg = ids_for_task("Rg")
ids_ffv = ids_for_task("FFV")
print("Tg ids:", ids_tg.shape, "Density ids:", ids_den.shape)

Tg ids: (511,) Density ids: (613,)


In [16]:
from torch.utils.data import Dataset
from torch_geometric.data import Data
import torch, numpy as np
from dataset_polymer_fixed import LMDBDataset

def _get_rdkit_feats_from_record(rec):
    arr = getattr(rec, "rdkit_feats", None)
    if arr is None:
        return torch.zeros(15, dtype=torch.float32)   # or 6 if that’s your build
    v = torch.as_tensor(np.asarray(arr, np.float32).reshape(-1), dtype=torch.float32)
    return v.unsqueeze(0)  # <<< IMPORTANT: (1, D) so batch -> (B, D)


class LMDBtoPyGSingleTask(Dataset):
    def __init__(
        self,
        ids,
        lmdb_path,
        target_index=None,
        *,
        use_mixed_edges: bool = True,      # <— enables 3 cat + 32 RBF continuous
        include_extra_atom_feats: bool = True,  # <— attach per-atom extras
    ):
        self.base = LMDBDataset(ids, lmdb_path)
        self.t = target_index
        self.use_mixed_edges = use_mixed_edges
        self.include_extra_atom_feats = include_extra_atom_feats

    def __len__(self): return len(self.base)

    def __getitem__(self, idx):
        rec = self.base[idx]

        x  = torch.as_tensor(rec.x, dtype=torch.long)
        ei = torch.as_tensor(rec.edge_index, dtype=torch.long)

        ea = torch.as_tensor(rec.edge_attr)              # (E, 3 + 32)
        if self.use_mixed_edges:
            # keep all columns; EdgeEncoderMixed will split cat vs cont
            edge_attr = ea.to(torch.float32)
        else:
            # categorical-only for vanilla BondEncoder
            edge_attr = ea[:, :3].to(torch.long)

        # rdkit globals: KEEP AS (1, D) so PyG collates to (B, D)
        rdkit_feats = _get_rdkit_feats_from_record(rec)  # (1, D)
        d = Data(x=x, edge_index=ei, edge_attr=edge_attr, rdkit_feats=rdkit_feats)

        if self.include_extra_atom_feats and hasattr(rec, "extra_atom_feats"):
            d.extra_atom_feats = torch.as_tensor(rec.extra_atom_feats, dtype=torch.float32)  # (N,5)

        if hasattr(rec, "has_xyz"):
            # collates to (B,1); handy as a gating/global indicator
            hz = np.asarray(rec.has_xyz, np.uint8).reshape(-1)
            d.has_xyz = torch.from_numpy(hz.astype(np.float32))

        if (self.t is not None) and hasattr(rec, "y"):
            yv = torch.as_tensor(rec.y, dtype=torch.float32).view(-1)
            if self.t < yv.numel():
                d.y = yv[self.t:self.t+1]  # (1,)

        # geometry & extras from LMDB (if present)
        if hasattr(rec, "pos"):              # (N,3) float
            d.pos = torch.as_tensor(rec.pos, dtype=torch.float32)
        if hasattr(rec, "extra_atom_feats"): # (N,5) float
            d.extra_atom_feats = torch.as_tensor(rec.extra_atom_feats, dtype=torch.float32)
        if hasattr(rec, "has_xyz"):          # (1,) bool/uint8
            d.has_xyz = torch.as_tensor(rec.has_xyz, dtype=torch.float32)
        # LMDBtoPyGSingleTask.__getitem__  (add this near the end, after you create Data d)
        if hasattr(rec, "dist"):
            # rec.dist is (L, L) (uint8) in your LMDB
            d.hops = torch.as_tensor(rec.dist, dtype=torch.long).unsqueeze(0)  # (1, L, L)

        return d

In [17]:
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader as GeoDataLoader

def make_loaders_for_task(task, ids, *, batch_size=64, seed=42,
                          use_mixed_edges=True, include_extra_atom_feats=True):
    t = task2idx[task]
    tr_ids, va_ids = train_test_split(ids, test_size=0.2, random_state=seed)
    tr_ds = LMDBtoPyGSingleTask(tr_ids, TRAIN_LMDB, target_index=t,
                                use_mixed_edges=use_mixed_edges,
                                include_extra_atom_feats=include_extra_atom_feats)
    va_ds = LMDBtoPyGSingleTask(va_ids, TRAIN_LMDB, target_index=t,
                                use_mixed_edges=use_mixed_edges,
                                include_extra_atom_feats=include_extra_atom_feats)
    tr = GeoDataLoader(tr_ds, batch_size=batch_size, shuffle=True,  num_workers=0, pin_memory=True)
    va = GeoDataLoader(va_ds, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    return tr, va


## Step 5: Define the Hybrid GNN Model

The final architecture uses both structural and cheminformatics data by combining GNN-learned graph embeddings with SMILES-derived RDKit descriptors. This Hybrid GNN model uses `smiles2graph` for graph construction and augments it with RDKit-based molecular features for improved prediction accuracy.

### Model Components:

* **AtomEncoder / BondEncoder**
  Transforms categorical atom and bond features (provided by OGB) into learnable embeddings using the encoders from `ogb.graphproppred.mol_encoder`. These provide a strong foundation for expressive graph learning.

* **GINEConv Layers (x2)**
  I use two stacked GINEConv layers (Graph Isomorphism Network with Edge features). These layers perform neighborhood aggregation based on edge attributes, allowing the model to capture localized chemical environments.

* **Global Mean Pooling**
  After message passing, node level embeddings are aggregated into a fixed size graph level representation using `global_mean_pool`.

* **Concatenation with RDKit Descriptors**
  The pooled GNN embedding is concatenated with external RDKit descriptors, which capture global molecular properties not easily inferred from graph data alone.

* **MLP Prediction Head**
  A multilayer perceptron processes the combined feature vector with ReLU activations, dropout regularization, and linear layers to predict the HOMO–LUMO gap.

In [18]:
import math, numpy as np, torch
from torch import nn
from torch.optim import AdamW, RMSprop
from torch.amp import GradScaler, autocast
from copy import deepcopy

def train_hybrid_gnn_sota(
    model: nn.Module,
    train_loader,
    val_loader,
    *,
    lr: float = 5e-4,
    optimizer: str = "AdamW",
    weight_decay: float = 1e-5,
    epochs: int = 120,
    warmup_epochs: int = 5,
    patience: int = 15,
    clip_norm: float = 1.0,
    amp: bool = True,
    loss_name: str = "mse",   # "mse" or "huber"
    save_dir: str = "saved_models/gnn",
    tag: str = "model_sota",
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
):
    import os
    os.makedirs(save_dir, exist_ok=True)
    model = model.to(device)

    # optimizer
    opt_name = optimizer.lower()
    if opt_name == "rmsprop":
        opt = RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=0.0)
    else:
        opt = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # cosine schedule w/ warmup
    def lr_factor(epoch):
        if epoch < warmup_epochs:
            return (epoch + 1) / max(1, warmup_epochs)
        t = (epoch - warmup_epochs) / max(1, (epochs - warmup_epochs))
        return 0.5 * (1 + math.cos(math.pi * t))
    scaler = GradScaler("cuda", enabled=amp)

    def loss_fn(pred, target):
        if loss_name.lower() == "huber":
            return F.huber_loss(pred, target, delta=1.0)
        return F.mse_loss(pred, target)

    @torch.no_grad()
    def eval_once(loader):
        model.eval()
        preds, trues = [], []
        for b in loader:
            b = b.to(device)
            p = model(b)
            preds.append(p.detach().cpu())
            trues.append(b.y.view(-1,1).cpu())
        preds = torch.cat(preds).numpy(); trues = torch.cat(trues).numpy()
        mae = np.mean(np.abs(preds - trues))
        rmse = float(np.sqrt(np.mean((preds - trues)**2)))
        r2 = float(1 - np.sum((preds - trues)**2) / np.sum((trues - trues.mean())**2))
        return mae, rmse, r2

    best_mae = float("inf")
    best = None
    best_path = os.path.join(save_dir, f"{tag}.pt")

    for ep in range(1, epochs+1):
        # schedule
        for g in opt.param_groups:
            g["lr"] = lr * lr_factor(ep-1)

        model.train()
        total, count = 0.0, 0
        for b in train_loader:
            b = b.to(device)
            with autocast("cuda", enabled=amp):
                pred = model(b)
                loss = loss_fn(pred, b.y.view(-1,1))

            opt.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            if clip_norm is not None:
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_norm)
            scaler.step(opt); scaler.update()

            total += loss.item() * b.num_graphs
            count += b.num_graphs

        tr_mse = total / max(1, count)
        mae, rmse, r2 = eval_once(val_loader)
        print(f"Epoch {ep:03d} | tr_MSE {tr_mse:.5f} | val_MAE {mae:.5f} | val_RMSE {rmse:.5f} | R2 {r2:.4f}")

        if mae < best_mae - 1e-6:
            best_mae = mae
            best = deepcopy(model.state_dict())
            torch.save(best, best_path)
            bad = 0
        else:
            bad += 1
            if bad >= patience:
                print("Early stopping.")
                break

    if best is not None:
        model.load_state_dict(best)
    else:
        model.load_state_dict(torch.load(best_path, map_location=device))

    final_mae, final_rmse, final_r2 = eval_once(val_loader)
    print(f"[{tag}] Best Val — MAE {final_mae:.6f} | RMSE {final_rmse:.6f} | R2 {final_r2:.4f}")
    return model, best_path, {"MAE": final_mae, "RMSE": final_rmse, "R2": final_r2}

In [19]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import to_dense_batch, to_dense_adj
from ogb.graphproppred.mol_encoder import AtomEncoder

# ---------- helpers ----------
class DropPath(nn.Module):
    def __init__(self, p=0.0): super().__init__(); self.p=float(p)
    def forward(self, x):
        if self.p==0.0 or not self.training: return x
        keep = 1.0 - self.p
        shape = (x.size(0),) + (1,)*(x.ndim-1)
        rnd = keep + torch.rand(shape, device=x.device, dtype=x.dtype)
        rnd.floor_()
        return x * rnd / keep

def _act(name): 
    name=(name or "relu").lower()
    return nn.GELU() if name=="gelu" else nn.SiLU() if name in ("swish","silu") else nn.ReLU()

# ---------- attention layer that accepts additive attn_bias ----------
class MHALayer(nn.Module):
    """
    Wraps torch.nn.MultiheadAttention but allows a per-head additive bias:
      attn_bias: (B, H, L, L) float, added to scaled dot-product before softmax.
    """
    def __init__(self, d_model, nhead, dropout=0.1):
        super().__init__()
        self.mha = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.nhead = nhead

    def forward(self, x, key_padding_mask, attn_bias):
        # x: (B,L,D), key_padding_mask: (B,L) True=PAD, attn_bias: (B,H,L,L)
        B, L, D = x.shape
        H = self.nhead
        # PyTorch accepts attn_mask shaped (B*H, L, L) additive
        mask_add = attn_bias.reshape(B*H, L, L)
        out, _ = self.mha(
            x, x, x,
            key_padding_mask=key_padding_mask,  # True at pads
            attn_mask=mask_add,
            need_weights=False
        )
        return out

# ---------- Transformer block (pre-norm, bias-aware attention, FFN, drop-path) ----------
class TFBlock(nn.Module):
    def __init__(self, d_model, nhead, mlp_ratio=4.0, act="gelu", dropout=0.1, drop_path=0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.attn  = MHALayer(d_model, nhead, dropout=dropout)
        self.dp1   = DropPath(drop_path)
        self.norm2 = nn.LayerNorm(d_model)
        hidden = int(d_model * mlp_ratio)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, hidden), _act(act), nn.Dropout(dropout),
            nn.Linear(hidden, d_model), nn.Dropout(dropout),
        )
        self.dp2   = DropPath(drop_path)

    def forward(self, x, key_padding_mask, attn_bias):
        h = self.attn(self.norm1(x), key_padding_mask, attn_bias)
        x = x + self.dp1(h)
        h = self.ffn(self.norm2(x))
        x = x + self.dp2(h)
        return x

# ---------- build geometry/graph attention bias ----------
class AttnBiasBuilder(nn.Module):
    def __init__(self, n_heads, rbf_k=32, dmin=0.0, dmax=10.0, beta=5.0, act="silu"):
        super().__init__()
        self.n_heads = n_heads
        self.register_buffer("centers", torch.linspace(dmin, dmax, rbf_k).view(1,1,rbf_k))
        self.beta = beta
        self.geo_mlp = nn.Sequential(
            nn.Linear(rbf_k, 2*n_heads), _act(act), nn.Linear(2*n_heads, n_heads)
        )
        self.adj_bias = nn.Parameter(torch.zeros(n_heads))
        nn.init.normal_(self.adj_bias, std=0.02)




    def forward(self, pos, edge_index, batch, key_padding_mask):
        """
        Build additive attention bias of shape (B, H, L0, L0) before CLS.
        key_padding_mask: (B, L0)   True == PAD   (no CLS here)
        Uses:
        - self.n_heads (int)
        - self.centers (K,)  float tensor for RBF centers
        - self.beta    (float) scalar for RBF width
        - self.geo_mlp : nn.Sequential(K -> H) maps RBF to per-head geometry bias
        - self.adj_bias: nn.Parameter(H,) per-head adjacency bias scale
        """
        # 1) dense adjacency  (B, L0, L0)
        A = to_dense_adj(edge_index, batch=batch)          # (B, 1, L0, L0) or (B, L0, L0) depending on version
        if A.dim() == 4:
            A = A.squeeze(1)                               # -> (B, L0, L0)

        B, L0, _ = A.shape
        H = self.n_heads
        device = A.device

        # 2) geometry bias (zeros if pos is None)
        if pos is not None:
            pad_pos, _valid = to_dense_batch(pos, batch)   # (B, L0, 3)
            # pairwise distances
            diff = pad_pos.unsqueeze(2) - pad_pos.unsqueeze(1)         # (B, L0, L0, 3)
            dist = torch.sqrt(torch.clamp((diff**2).sum(-1), min=0.0)) # (B, L0, L0)

            centers = self.centers.to(dist.device)  # (K,)
            rbf = torch.exp(-self.beta * (dist.unsqueeze(-1) - centers)**2)  # (B, L0, L0, K)
            geo_bias = self.geo_mlp(rbf).permute(0, 3, 1, 2).contiguous()    # (B, H, L0, L0)
            geo_bias = geo_bias.to(torch.float32)
        else:
            geo_bias = torch.zeros((B, H, L0, L0), device=device, dtype=torch.float32)

        # 3) adjacency bias per head
        #     A is (B, L0, L0) float; scale each head with learned scalar
        adj_bias = self.adj_bias.view(1, H, 1, 1) * A.unsqueeze(1)     # (B, H, L0, L0)

        # 4) PAD masking (rows from PAD, cols to PAD) with big negative
        #    (we keep diagonal as-is here; model will final-fix diagonals if needed)
        pad = key_padding_mask.to(torch.bool)                           # (B, L0)
        big_neg = geo_bias.new_tensor(-1e4)

        # rows FROM PAD
        geo_bias = geo_bias.masked_fill(pad.view(B, 1, L0, 1), big_neg)
        adj_bias = adj_bias.masked_fill(pad.view(B, 1, L0, 1), 0.0)
        # cols TO PAD
        geo_bias = geo_bias.masked_fill(pad.view(B, 1, 1, L0), big_neg)
        adj_bias = adj_bias.masked_fill(pad.view(B, 1, 1, L0), 0.0)

        # 5) return head-wise additive bias
        return geo_bias + adj_bias                                     # (B, H, L0, L0)

# ---------- the Graph Transformer ----------
class GraphTransformer(nn.Module):
    def __init__(
        self,
        *,
        d_model: int = 256,
        nhead: int = 8,
        nlayers: int = 6,
        mlp_ratio: float = 4.0,
        dropout: float = 0.1,
        drop_path: float = 0.1,
        atom_emb_dim: int = 256,          # kept equal to d_model
        rdkit_dim: int = 15,              # from LMDB (6 or 15)
        extra_atom_dim: int = 5,          # LMDB extra per-atom feats
        activation: str = "silu",
        use_extra_atom_feats: bool = True,
        use_cls_token: bool = True,
        use_has_xyz: bool = True,
    ):
        super().__init__()
        self.use_extra_atom_feats = use_extra_atom_feats
        self.use_cls_token = use_cls_token
        self.use_has_xyz = use_has_xyz
        self.d_model = d_model

        self.atom_enc = AtomEncoder(emb_dim=atom_emb_dim)
        if use_extra_atom_feats:
            self.extra_proj = nn.Sequential(
                nn.Linear(extra_atom_dim, d_model), _act(activation),
                nn.Linear(d_model, d_model)
            )
            self.extra_gate = nn.Sequential(
                nn.Linear(2*d_model, d_model), _act(activation)
            )

        # class token appended at end of each sequence
        if use_cls_token:
            self.cls_token = nn.Parameter(torch.zeros(1, d_model))
            nn.init.normal_(self.cls_token, std=0.02)

        # blocks with linearly increasing drop-path
        dprs = [drop_path * i / max(1,(nlayers-1)) for i in range(nlayers)]
        self.blocks = nn.ModuleList([
            TFBlock(d_model, nhead, mlp_ratio, activation, dropout, dprs[i]) for i in range(nlayers)
        ])

        self.bias_builder = AttnBiasBuilder(n_heads=nhead, rbf_k=32, dmin=0.0, dmax=10.0, beta=5.0, act=activation)

        head_in = d_model + rdkit_dim + (1 if use_has_xyz else 0)
        self.head = nn.Sequential(
            nn.LayerNorm(head_in),
            nn.Linear(head_in, 2*d_model), _act(activation), nn.Dropout(dropout),
            nn.Linear(2*d_model, d_model//2), _act(activation), nn.Dropout(dropout),
            nn.Linear(d_model//2, 1),
        )

    def forward(self, data):
        x = self.atom_enc(data.x)
        if self.use_extra_atom_feats and hasattr(data, "extra_atom_feats"):
            xa = self.extra_proj(data.extra_atom_feats)
            x  = self.extra_gate(torch.cat([x, xa], dim=1))

        # Pack (no CLS yet)
        x_pad, valid_mask = to_dense_batch(x, data.batch)  # (B,L0,D)
        B, L0, D = x_pad.shape
        key_padding_mask = ~valid_mask                     # (B,L0)  True=PAD

        # Build bias on L0 tokens (no CLS)
        attn_bias = self.bias_builder(
            pos=data.pos if hasattr(data, "pos") else None,
            edge_index=data.edge_index, batch=data.batch,
            key_padding_mask=key_padding_mask
        )  # (B,H,L0,L0)

        # Append CLS and pad bias
        if self.use_cls_token:
            cls = self.cls_token.expand(B, 1, D)
            x_pad = torch.cat([x_pad, cls], dim=1)                        # (B,L0+1,D)
            key_padding_mask = torch.cat(
                [key_padding_mask, torch.zeros(B,1, dtype=torch.bool, device=x_pad.device)],
                dim=1
            )                                                              # (B,L0+1)
            # pad attn_bias by one row and one col (zeros) for CLS
            attn_bias = F.pad(attn_bias, (0,1, 0,1))                       # (B,H,L0+1,L0+1)
            L = L0 + 1
        else:
            L = L0

        # Transformer
        h = x_pad
        for blk in self.blocks:
            h = blk(h, key_padding_mask[:, :L], attn_bias)

        # Readout
        if self.use_cls_token:
            gvec = h[:, L-1, :]
        else:
            gvec = (h * valid_mask.unsqueeze(-1)).sum(1) / valid_mask.sum(1, keepdim=True)

        rd = data.rdkit_feats.view(B, -1).float()
        pieces = [gvec, rd]
        if self.use_has_xyz and hasattr(data, "has_xyz"):
            pieces.append(data.has_xyz.view(B,1).float())
        out = torch.cat(pieces, dim=1)
        return self.head(out)

In [20]:
# Build loaders with mixed edges ON (so edge_attr keeps the 3 cat + 32 RBF),
# and with extra_atom_feats so the model can use them.
train_loader_tg,  val_loader_tg  = make_loaders_for_task("Tg",      ids_tg,  batch_size=512,
                                                         use_mixed_edges=True, include_extra_atom_feats=True)
train_loader_den, val_loader_den = make_loaders_for_task("Density", ids_den, batch_size=512,
                                                         use_mixed_edges=True, include_extra_atom_feats=True)
train_loader_rg,  val_loader_rg  = make_loaders_for_task("Rg",      ids_rg,  batch_size=512,
                                                         use_mixed_edges=True, include_extra_atom_feats=True)



In [21]:
# infer rdkit_dim from a batch (6 or 15 depending on your LMDB)
b = next(iter(train_loader_tg))
rd_dim = int(b.rdkit_feats.shape[-1])

model_tg = GraphTransformer(d_model=256, nhead=8, nlayers=6,
                            dropout=0.2, drop_path=0.1,
                            rdkit_dim=rd_dim, use_extra_atom_feats=True,
                            activation="silu").to(b.x.device)

model_tg, ckpt_tg, metrics_tg = train_hybrid_gnn_sota(
    model_tg, train_loader_tg, val_loader_tg,
    lr=6e-4, optimizer="AdamW", weight_decay=1e-5,
    epochs=120, warmup_epochs=5, patience=20,
    clip_norm=1.0, amp=True, loss_name="mse",
    save_dir="saved_models/gt_tg", tag="graphtransformer_tg"
)


# Density (use your tuned dims if you like larger backbones)
model_den = GraphTransformer(d_model=256,nhead=8, nlayers=6,
                             dropout=0.2, drop_path=0.1,
                             rdkit_dim=rd_dim, use_extra_atom_feats=True,
                             activation="silu").to(b.x.device)

model_den, ckpt_den, metrics_den = train_hybrid_gnn_sota(
    model_den, train_loader_den, val_loader_den,
    lr=6e-4, optimizer="AdamW", weight_decay=1e-5,
    epochs=200, warmup_epochs=8, patience=30,
    clip_norm=1.0, amp=True, loss_name="mse",
    save_dir="saved_models/gt_density", tag="graphtransformer_density"
)


# Rg (your tuned gnn_dim + swish + RMSprop work fine here)
model_rg = GraphTransformer(d_model=256, nhead=8, nlayers=6,
                            dropout=0.2, drop_path=0.1,
                            rdkit_dim=rd_dim, use_extra_atom_feats=True,
                            activation="silu").to(b.x.device)

model_rg, ckpt_rg, metrics_rg = train_hybrid_gnn_sota(
    model_rg, train_loader_rg, val_loader_rg,
    lr=6e-4, optimizer="AdamW", weight_decay=1e-5,
    epochs=200, warmup_epochs=6, patience=20,
    clip_norm=1.0, amp=True, loss_name="huber",  # Huber often helps Rg
    save_dir="saved_models/gt_rg", tag="gt_rg"
)


c:\Users\mattg\anaconda3\envs\chemml_env\lib\site-packages\torch\nn\functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
c:\Users\mattg\anaconda3\envs\chemml_env\lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch 001 | tr_MSE 22792.74805 | val_MAE 100.59009 | val_RMSE 131.10902 | R2 -0.8007
Epoch 002 | tr_MSE 22792.79492 | val_MAE 100.59009 | val_RMSE 131.10902 | R2 -0.8007
Epoch 003 | tr_MSE 22792.75000 | val_MAE 100.59009 | val_RMSE 131.10902 | R2 -0.8007
Epoch 004 | tr_MSE 22792.78320 | val_MAE 100.59009 | val_RMSE 131.10902 | R2 -0.8007
Epoch 005 | tr_MSE 22793.02539 | val_MAE 100.59009 | val_RMSE 131.10902 | R2 -0.8007
Epoch 006 | tr_MSE 22793.52344 | val_MAE 100.59009 | val_RMSE 131.10902 | R2 -0.8007
Epoch 007 | tr_MSE 22792.68555 | val_MAE 100.59009 | val_RMSE 131.10902 | R2 -0.8007
Epoch 008 | tr_MSE 22793.31445 | val_MAE 100.59009 | val_RMSE 131.10902 | R2 -0.8007
Epoch 009 | tr_MSE 22791.89062 | val_MAE 100.42054 | val_RMSE 130.91714 | R2 -0.7954
Epoch 010 | tr_MSE 22738.57227 | val_MAE 100.25423 | val_RMSE 130.73039 | R2 -0.7903
Epoch 011 | tr_MSE 22684.26367 | val_MAE 100.08749 | val_RMSE 130.54160 | R2 -0.7851
Epoch 012 | tr_MSE 22630.24023 | val_MAE 99.90849 | val_RMSE 130.

# Conclusions

## Model Performance Summary

All baseline models were initially trained and evaluated on a 5,000 molecule subset of the full dataset. Below is a comparison of results across different featurization strategies and model types:

### 2D Baseline Models

| Model Type    | Featurization      | MAE   | RMSE  | R²    | Notes                                 |
| ------------- | ------------------ | ----- | ----- | ----- | ------------------------------------- |
| MLP (Tuned)   | RDKit Fingerprints | 0.426 | 0.574 | 0.798 | Strong performance across all metrics |
| KRR (Tuned)   | RDKit Fingerprints | 0.454 | 0.593 | 0.784 | Good overall, slightly lower R²       |
| RF (Tuned)    | RDKit Fingerprints | 0.423 | 0.583 | 0.791 | Best MAE, very competitive overall    |
| MLP (Tuned)   | Coulomb Matrix     | 0.636 | 0.819 | 0.588 | Significantly weaker performance      |
| MLP (Untuned) | RDKit Fingerprints | 0.467 | 0.609 | 0.772 | Solid untuned baseline                |
| KRR (Untuned) | RDKit Fingerprints | 0.519 | 0.668 | 0.726 | Notable drop from tuned version       |
| RF (Untuned)  | RDKit Fingerprints | 0.426 | 0.587 | 0.788 | Surprisingly close to tuned RF        |
| MLP (Untuned) | Coulomb Matrix     | 0.663 | 0.847 | 0.559 | Consistently underperforms            |

### Graph Neural Network Models (ChemML)

| Model Type    | Featurization               | MAE   | RMSE  | R²    | Notes                                |
| ------------- | --------------------------- | ----- | ----- | ----- | ------------------------------------ |
| GNN (Tuned)   | `tensorise_molecules` Graph | 0.302 | 0.411 | 0.900 | Best results from ChemML experiments |
| GNN (Untuned) | `tensorise_molecules` Graph | 0.400 | 0.519 | 0.841 | Strong but less optimized            |

### Final Hybrid GNN Model Trained on Full Dataset (OGB-Compatible)

| Model Type           | Featurization                          | MAE   | RMSE  | R²    | Notes                              |
| -------------------- | -------------------------------------- | ----- | ----- | ----- | ---------------------------------- |
| Hybrid GNN (Tuned)   | OGB `smiles2graph` + RDKit descriptors | 0.159 | 0.234 | 0.965 | State-of-the-art level performance |
| Hybrid GNN (Untuned) | OGB `smiles2graph` + RDKit descriptors | 0.223 | 0.308 | 0.939 | Still very strong pre-tuning       |

---

## Model Error Analysis

I performed qualitative evaluation by comparing predicted vs. true HOMO–LUMO gaps for both randomly selected and poorly predicted molecules. The worst performing molecules often showed rare or complex structures likely underrepresented in the training set. This highlights the importance of structural diversity and potentially more expressive 3D information to improve generalization.

## Next Steps: Integrating 3D Molecular Information

To push performance even further and overcome limitations of 2D graphs and hand crafted descriptors, my next step will involve:

* Using **3D molecular geometries** 
* Incorporating **interatomic distances**, angles, and **spatial encoding** (SchNet, DimeNet, or SE(3)-equivariant models)
* Comparing results against the current best MAE (\~0.159)

This direction aligns with trends in molecular property prediction where 3D aware models often outperform purely 2D approaches, especially for quantum properties like HOMO–LUMO gaps.
